# Importing data and lemmatizing the data

In [ ]:
import numpy as np
import pandas as pd
import re
import scipy
import math

In [ ]:
URL = 'https://drive.google.com/file/d/137eW4F35OctoRuq5DasVUXw6GpmfXdBS/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+URL.split('/')[-2]
#df = pd.read_pickle(path)
data = pd.read_csv(path, skip_blank_lines=True)
pd.set_option('display.max_colwidth', None)
print(data.shape)

(1950, 6)


In [ ]:
data.drop_duplicates(subset='content', inplace=True, ignore_index=True)
data.shape

(1761, 6)

In [ ]:
data.head(1)

Unnamed: 0                       _id  ...   topics     date_pub
0           0  6076fad70b3e8bc9b779293a  ...  cricket  Apr 4, 2021

[1 rows x 6 columns]

In [ ]:
data[data['_id'] == '6076fadb0b3e8bc9b779293e']['_id'].to_string()

'4    6076fadb0b3e8bc9b779293e'

In [ ]:
def make_lower_case(text):
    return text.lower()

In [ ]:
import re
from pprint import pprint
import nltk, spacy, gensim
from sklearn.feature_extraction.text import CountVectorizer

def get_lemmatized_clean_data(df):
    # Convert to list
    data = df.content.tolist()

    # Remove Emails
    data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

    # Remove new line characters
    data = [re.sub('\s+', ' ', sent) for sent in data]

    # Remove distracting single quotes
    data = [re.sub("\'", "", sent) for sent in data]

    # pprint(data[:1])

    def sent_to_words(sentences):
        for sentence in sentences:
            yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

    data_words = list(sent_to_words(data))

    # print(data_words[:1])
    
    def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
        """https://spacy.io/api/annotation"""
        texts_out = []
        for sent in texts:
            doc = nlp(" ".join(sent)) 
            texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
        return texts_out

    # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
    # Run in terminal: python3 -m spacy download en
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

    # Do lemmatization keeping only Noun, Adj, Verb, Adverb
    data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

    return data_lemmatized

In [ ]:
X = get_lemmatized_clean_data(data)

In [ ]:
max_time = []
for i in X:
  max_time.append(len(i.split(' '))/2.5)

data['Max_Time'] = max_time

In [ ]:
data.head()

# SKlearn NewsData Import

In [ ]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')

In [ ]:
def get_data(mydata):
    mydata.keys()
    df = pd.DataFrame([mydata['data'],[mydata['target_names'][idx] for idx in mydata['target']],mydata['target']])
    df = df.transpose()
    df.columns = ['content', 'target_names', 'target']
    return df
    
df = get_data(newsgroups_train)

In [ ]:
df.head()

In [ ]:
news = data.drop(axis = 1, columns=['_id', 'y',]).to_numpy()

In [ ]:
data_lemmatized = get_lemmatized_clean_data(df)

In [ ]:
df.head()

,content,target_names,target
0,"From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n",rec.autos,7
1,"From: guykuo@carson.u.washington.edu (Guy Kuo)\nSubject: SI Clock Poll - Final Call\nSummary: Final call for SI clock reports\nKeywords: SI,acceleration,clock,upgrade\nArticle-I.D.: shelley.1qvfo9INNc3s\nOrganization: University of Washington\nLines: 11\nNNTP-Posting-Host: carson.u.washington.edu\n\nA fair number of brave souls who upgraded their SI clock oscillator have\nshared their experiences for this poll. Please send a brief message detailing\nyour experiences with the procedure. Top speed attained, CPU rated speed,\nadd on cards and adapters, heat sinks, hour of usage per day, floppy disk\nfunctionality with 800 and 1.4 m floppies are especially requested.\n\nI will be summarizing in the next two days, so please add to the network\nknowledge base if you have done the clock upgrade and haven't answered this\npoll. Thanks.\n\nGuy Kuo <guykuo@u.washington.edu>\n",comp.sys.mac.hardware,4
2,"From: twillis@ec.ecn.purdue.edu (Thomas E Willis)\nSubject: PB questions...\nOrganization: Purdue University Engineering Computer Network\nDistribution: usa\nLines: 36\n\nwell folks, my mac plus finally gave up the ghost this weekend after\nstarting life as a 512k way back in 1985. sooo, i'm in the market for a\nnew machine a bit sooner than i intended to be...\n\ni'm looking into picking up a powerbook 160 or maybe 180 and have a bunch\nof questions that (hopefully) somebody can answer:\n\n* does anybody know any dirt on when the next round of powerbook\nintroductions are expected? i'd heard the 185c was supposed to make an\nappearence ""this summer"" but haven't heard anymore on it - and since i\ndon't have access to macleak, i was wondering if anybody out there had\nmore info...\n\n* has anybody heard rumors about price drops to the powerbook line like the\nones the duo's just went through recently?\n\n* what's the impression of the display on the 180? i could probably swing\na 180 if i got the 80Mb disk rather than the 120, but i don't really have\na feel for how much ""better"" the display is (yea, it looks great in the\nstore, but is that all ""wow"" or is it really that good?). could i solicit\nsome opinions of people who use the 160 and 180 day-to-day on if its worth\ntaking the disk size and money hit to get the active display? (i realize\nthis is a real subjective question, but i've only played around with the\nmachines in a computer store breifly and figured the opinions of somebody\nwho actually uses the machine daily might prove helpful).\n\n* how well does hellcats perform? ;)\n\nthanks a bunch in advance for any info - if you could email, i'll post a\nsummary (news reading time is at a premium with finals just around the\ncorner... :( )\n--\nTom Willis \ twillis@ecn.purdue.edu \ Purdue Electrical Engineering\n---------------------------------------------------------------------------\n""Convictions are more dangerous enemies of truth than lies."" - F. W.\nNietzsche\n",comp.sys.mac.hardware,4
3,"From: jgreen@amber (Joe Green)\nSubject: Re: Weitek P9000 ?\nOrganization: Harris Computer Systems Division\nLines: 14\nDistribution: world\nNNTP-Posting-Host: amber.ssd.csd.harris.com\nX-Newsreader: TIN [version 1.1 PL9]\n\nRobert J.C. Kyanko (rob@rjck.UUCP)

# Converting the data to bag_of_word representation

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
my_stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD

In [ ]:
vectorizor = TfidfVectorizer(stop_words=my_stopwords, lowercase= True)
bag_of_words = vectorizor.fit_transform(X)

# Content Based Similarity Using TFIDF Vector

In [ ]:
from numpy import dot
from numpy.linalg import norm

def similarity(a,b):
  cos_sim = dot(a, b)/(norm(a)*norm(b))
  return cos_sim

In [ ]:
def ContentBasedFiltering(id, first_n = 10):
  similarity_dic = {}
  news_index = data[data['_id']==id].index[0]
  for i in data['_id']:
    an_index = data[data['_id']==i].index[0]
    a = np.array(bag_of_words[news_index].todense())[0]
    b = np.array(bag_of_words[an_index].todense())[0]
    similarity_dic[i] = similarity(a, b)

  sorted_most_similar = sorted(similarity_dic.items(), key = 
             lambda kv:(kv[1], kv[0]), reverse=True)
  
  return sorted_most_similar[:first_n]

ContentBasedFiltering('6076fadb0b3e8bc9b779293e')


[('6076fadb0b3e8bc9b779293e', 1.0000000000000002),
 ('6076fb580b3e8bc9b7792991', 0.3649237822888533),
 ('6076fbe10b3e8bc9b77929ec', 0.306834385284666),
 ('6077053f0b3e8bc9b7793016', 0.23287259673719873),
 ('6076fbef0b3e8bc9b77929f5', 0.17687503497034085),
 ('6076fced0b3e8bc9b7792a9f', 0.17006402697253414),
 ('6076ff750b3e8bc9b7792c41', 0.163074639403999),
 ('607702d20b3e8bc9b7792e76', 0.1615345680899587),
 ('6076ff790b3e8bc9b7792c44', 0.16018104823447651),
 ('6077056b0b3e8bc9b7793032', 0.1579443125007366)]

In [ ]:
for keys in ContentBasedFiltering('6076fadb0b3e8bc9b779293e'):
  print(data[data['_id'] == keys[0]]['title'])

4    Brisbane is the 'preferred host' for 2032 Olympics, IOC announces
Name: title, dtype: object
86    The Olympic flame starts its final leg to Tokyo. Some suggest this day should never have come
Name: title, dtype: object
177    Pulling off Tokyo 2020 will be a logistical nightmare ... and the clock is ticking
Name: title, dtype: object
1605    MLB All-Star Game Fast Facts
Name: title, dtype: object
186    Australian Open turmoil raises questions for Tokyo Games
Name: title, dtype: object
349    Premier League facing backlash over $19 pay-per-view charge
Name: title, dtype: object
731    August 7 coronavirus news
Name: title, dtype: object
1285    Bloomberg and The Washington Post are betting big on gaming. Here's why
Name: title, dtype: object
734    July 30 coronavirus news
Name: title, dtype: object
1627    Texas Rangers sell over 38,000 tickets to home opener, marking one of the first full-capacity sporting events in a year
Name: title, dtype: object


# Content Based Similarity Using SVD


In [ ]:
# Performing SVD

svd = TruncatedSVD(n_components=50)
lsa = svd.fit_transform(bag_of_words)

In [ ]:
def SVDContentBasedFiltering(id, first_n = 10):
  similarity_dic = {}
  news_index = data[data['_id']==id].index[0]
  for i in data['_id']:
    an_index = data[data['_id']==i].index[0]
    a = np.array(lsa[news_index])
    b = np.array(lsa[an_index])
    similarity_dic[i] = similarity(a, b)

  sorted_most_similar = sorted(similarity_dic.items(), key = 
             lambda kv:(kv[1], kv[0]), reverse=True)
  
  return sorted_most_similar[:first_n]

SVDContentBasedFiltering('6076fadb0b3e8bc9b779293e')

[('6076fadb0b3e8bc9b779293e', 1.0),
 ('6076fb580b3e8bc9b7792991', 0.8833394087381449),
 ('6077056b0b3e8bc9b7793032', 0.8523443951807746),
 ('607702d20b3e8bc9b7792e76', 0.8263748808275285),
 ('6076fbfb0b3e8bc9b77929fc', 0.8149561303414854),
 ('6076fced0b3e8bc9b7792a9f', 0.7550128419018863),
 ('6076fbe10b3e8bc9b77929ec', 0.7505506073390191),
 ('6076fde30b3e8bc9b7792b3f', 0.7464959618347344),
 ('6077053f0b3e8bc9b7793016', 0.7344521653903384),
 ('607705fb0b3e8bc9b7793091', 0.7261395314440704)]

In [ ]:
for keys in SVDContentBasedFiltering('6076fadb0b3e8bc9b779293e'):
  print(data[data['_id'] == keys[0]]['title'])

4    Brisbane is the 'preferred host' for 2032 Olympics, IOC announces
Name: title, dtype: object
86    The Olympic flame starts its final leg to Tokyo. Some suggest this day should never have come
Name: title, dtype: object
1627    Texas Rangers sell over 38,000 tickets to home opener, marking one of the first full-capacity sporting events in a year
Name: title, dtype: object
1285    Bloomberg and The Washington Post are betting big on gaming. Here's why
Name: title, dtype: object
193    James Naismith: Basketball inventor is celebrated with Google Doodle
Name: title, dtype: object
349    Premier League facing backlash over $19 pay-per-view charge
Name: title, dtype: object
177    Pulling off Tokyo 2020 will be a logistical nightmare ... and the clock is ticking
Name: title, dtype: object
505    Eight months pregnant athlete clinches Taekwondo gold medal
Name: title, dtype: object
1605    MLB All-Star Game Fast Facts
Name: title, dtype: object
1697    Baseball's Opening Day reflects a

# LDA Implementation


In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV
lda = LatentDirichletAllocation(learning_method='batch', n_jobs=-1)

In [ ]:
bag_of_words.T

<25658x1761 sparse matrix of type '<class 'numpy.float64'>'
	with 503616 stored elements in Compressed Sparse Column format>

In [ ]:
# LDA Cross-Validation
n_components = [20, 50, 70]
learning_decay = [0.5, 0.7, 0.8]


params = {'n_components': n_components, 'learning_decay': learning_decay}

model = GridSearchCV(lda, param_grid=params)
model.fit(bag_of_words.T)

GridSearchCV(cv=None, error_score=nan,
             estimator=LatentDirichletAllocation(batch_size=128,
                                                 doc_topic_prior=None,
                                                 evaluate_every=-1,
                                                 learning_decay=0.7,
                                                 learning_method='batch',
                                                 learning_offset=10.0,
                                                 max_doc_update_iter=100,
                                                 max_iter=10,
                                                 mean_change_tol=0.001,
                                                 n_components=10, n_jobs=-1,
                                                 perp_tol=0.1,
                                                 random_state=None,
                                                 topic_word_prior=None,
                                                 total_

In [ ]:
best_params = model.best_estimator_
best_params

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=20, n_jobs=-1,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)

In [ ]:
lda_res = best_params.components_.T

In [ ]:
lda_res.shape

(1761, 20)

In [ ]:
import pickle

pickle_file = 'lda_cross_validation_rev.pkl'

with open(pickle_file, 'wb') as file:  
    pickle.dump(model, file)

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)  

# get the folder id where you want to save your file
file = drive.CreateFile({'parents':[{u'id': '19AI35wfuabh1JQ6b1Z3YH5uJ6uL3N6BD'}]})
file.SetContentFile(pickle_file)
file.Upload() 

In [ ]:
with open(pickle_file, 'rb') as file:  
    lda_pkl_model = pickle.load(file)

In [ ]:
def LDAContentBasedFiltering(id, first_n = 10):
  similarity_dic = {}
  news_index = data[data['_id']==id].index[0]
  for i in data['_id']:
    an_index = data[data['_id']==i].index[0]
    a = np.array(lda_res[news_index])
    b = np.array(lda_res[an_index])
    similarity_dic[i] = similarity(a, b)

  sorted_most_similar = sorted(similarity_dic.items(), key = 
             lambda kv:(kv[1], kv[0]), reverse=True)
  
  return sorted_most_similar[:first_n]

LDAContentBasedFiltering('6076fadb0b3e8bc9b779293e')

[('6076fadb0b3e8bc9b779293e', 1.0),
 ('6077018d0b3e8bc9b7792da3', 0.9999996879823945),
 ('607703310b3e8bc9b7792eb8', 0.9999972308665471),
 ('607700e40b3e8bc9b7792d32', 0.9999966228645889),
 ('607705200b3e8bc9b7793002', 0.999988768170032),
 ('607702e90b3e8bc9b7792e87', 0.9999882506582742),
 ('607701780b3e8bc9b7792d93', 0.9999875745179311),
 ('6077032f0b3e8bc9b7792eb6', 0.9999866053029736),
 ('607701a90b3e8bc9b7792db5', 0.9999858718243335),
 ('607705f70b3e8bc9b779308d', 0.999985168949285)]

In [ ]:
for keys in LDAContentBasedFiltering('6076fadb0b3e8bc9b779293e'):
  print(data[data['_id'] == keys[0]]['title'])

4    Brisbane is the 'preferred host' for 2032 Olympics, IOC announces
Name: title, dtype: object
1079    Maryland inmates can now earn a bachelor's degree from Georgetown University
Name: title, dtype: object
1339    FEMA overwhelmed by calls on first day to apply for $9,000 for Covid-19 funeral expenses
Name: title, dtype: object
972    Alphabet employees demand company do more to stop harassment at work
Name: title, dtype: object
1592    Elena Kagan Fast Facts
Name: title, dtype: object
1301    Biden will address a joint session of Congress on April 28
Name: title, dtype: object
1065    There's a fix for what ails the Mars helicopter's software
Name: title, dtype: object
1337    Senate Democrats push bill to combat surge of Asian American hate crimes amid GOP resistance
Name: title, dtype: object
1093    Biden administration launches task force to ensure scientific decisions are free from political influence
Name: title, dtype: object
1693    Salman Rushdie reflects on post-colonial

# Word embedding using Glove Vectorizor

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()

glove_file = open('/content/drive/MyDrive/NLP and Text Analysis/glove.6B/glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

In [ ]:
i = 0
glov_stop = []
news_embedding_dict = dict()
for word in vectorizor.vocabulary_.keys():
  if word in embeddings_dictionary:
    news_embedding_dict[word] = embeddings_dictionary[word]
  else:
    glov_stop.append(word)


In [ ]:
stopset = set(nltk.corpus.stopwords.words('english'))
new_stopwords_list = stopset.union(glov_stop)
vectorizor_glov = TfidfVectorizer(stop_words=new_stopwords_list)

In [ ]:
glov_bag_of_words = vectorizor_glov.fit_transform(X)

In [ ]:
y = np.array([val for (key, val) in news_embedding_dict.items()])

In [ ]:
y.shape

(23233, 100)

In [ ]:
glov_bag_of_words.shape

(1761, 23233)

In [ ]:
document_embedding = glov_bag_of_words*y

In [ ]:
document_embedding.shape

(1761, 100)

In [ ]:
document_embedding

array([[-0.8209928 ,  0.45994018,  0.63383434, ..., -1.28062021,
         0.60099754,  1.10101666],
       [ 0.54033955,  0.26890623,  1.38927738, ..., -1.22477319,
        -0.68539252,  1.04541822],
       [-0.16363021,  0.10444771,  0.6272137 , ..., -1.38510202,
         1.03627766,  1.12584735],
       ...,
       [-0.38511505, -0.12654721,  1.01234692, ..., -1.24210309,
         0.52297703,  1.11222464],
       [-0.50131174,  0.5663506 ,  0.21643107, ..., -1.28896963,
        -0.04318757,  0.33516438],
       [ 0.2529258 ,  1.05304587,  1.32861671, ..., -1.94857593,
         0.01529223,  1.96337545]])

In [ ]:
def ContentBasedFilteringWordEmbedding(id, first_n = 10):
  similarity_dic = {}
  news_index = data[data['_id']==id].index[0]
  for i in data['_id']:
    an_index = data[data['_id']==i].index[0]
    a = np.array(document_embedding[news_index])
    b = np.array(document_embedding[an_index])
    similarity_dic[i] = similarity(a, b)

  sorted_most_similar = sorted(similarity_dic.items(), key = 
             lambda kv:(kv[1], kv[0]), reverse=True)
  
  return sorted_most_similar[:first_n]

ContentBasedFilteringWordEmbedding('6076fadb0b3e8bc9b779293e')

[('6076fadb0b3e8bc9b779293e', 1.0),
 ('6076fb580b3e8bc9b7792991', 0.9110987343021498),
 ('607705ca0b3e8bc9b7793070', 0.9023920302902176),
 ('6076ff710b3e8bc9b7792c3d', 0.9010180373827772),
 ('6076fb280b3e8bc9b779296f', 0.899418176608577),
 ('607702710b3e8bc9b7792e39', 0.8991524300806085),
 ('6076fbe10b3e8bc9b77929ec', 0.897788593871919),
 ('6077000a0b3e8bc9b7792ca2', 0.8972956336281251),
 ('607706190b3e8bc9b77930a4', 0.8961364239346054),
 ('607702150b3e8bc9b7792dfd', 0.8953714977588201)]

# Collaborative Filtering

In [ ]:
topics = ['cricket', 'football', 'golf', 'asia', 'africa', 'europe', 'americas', 'style', 'tech', 'science', 'hollywood', 'us politics', 'stock market', 'travel', 'coronavirus', 'black lives matter']
from random import sample
class User:
  def __init__(self, id):
    self.id = id
    self.prefered_categories = sample(topics, np.random.randint(low=3, high= 5))
    self.no_of_articles_served = np.random.randint(10, 50)*10
    self.no_of_sessions = math.ceil((self.no_of_articles_served)/10)
    self.ids = [self.id for _ in range(self.no_of_articles_served)]
    self.sessions = []
    self.articles_served = []
    self.ratings = []
    self.click = []
    self.ranks = []
    j = math.ceil(self.no_of_articles_served*0.7)
    for m in range(j):
      id_temp = np.random.choice(data[data['topics'].isin(self.prefered_categories)]['_id'])
      self.articles_served.append(id_temp)
      click = np.random.binomial(1, 0.7,1)[0]
      self.click.append(click)
      self.ratings.append('-' if click == 0 else np.random.randint((data[data['_id'] == id_temp]['Max_Time'])/4, data[data['_id'] == self.articles_served[m]]['Max_Time'])[0])

    j = self.no_of_articles_served-j
    for m in range(j):
      id_temp = np.random.choice(data[~data['topics'].isin(self.prefered_categories)]['_id'])
      self.articles_served.append(id_temp)
      click = np.random.binomial(1, 0.1,1)[0]
      self.click.append(click)
      self.ratings.append('-' if click == 0 else np.random.randint(0, data[data['_id'] == id_temp]['Max_Time'])[0])
    for i in range(self.no_of_sessions):
      for k in range(10):
        self.sessions.append(i)
        self.ranks.append(k)


In [ ]:
new_user = User(1)
data[data['_id'].isin(new_user.articles_served)].tail(10)

In [ ]:
def CreateRandomUserProfiler(max_no_user = 40):
  Users = []
  for i in range(max_no_user):
    Users.append(User(i))
    print(Users[i-1].prefered_categories)
    
  
  UserProfiler = pd.DataFrame(columns=['UserId', 'SessionID', 'ArticleID Served', 'Article Rank', 'Click', 'Time Spent'])
  for user in Users:
    df = pd.DataFrame()
    df['UserId'] = user.ids
    df['SessionID'] = user.sessions
    df['ArticleID Served'] = user.articles_served
    df['Article Rank'] = user.ranks
    df['Click'] = user.click
    df['Time Spent'] = user.ratings
    UserProfiler = pd.concat([UserProfiler,df], ignore_index=True)

  return UserProfiler

In [ ]:
UserProfiler = CreateRandomUserProfiler(40)

['football', 'hollywood', 'asia', 'coronavirus']
['football', 'hollywood', 'asia', 'coronavirus']
['travel', 'coronavirus', 'golf', 'europe']
['black lives matter', 'asia', 'africa', 'hollywood']
['americas', 'travel', 'black lives matter', 'europe']
['football', 'asia', 'us politics']
['europe', 'stock market', 'black lives matter']
['asia', 'style', 'europe']
['cricket', 'golf', 'travel']
['americas', 'golf', 'stock market', 'hollywood']
['hollywood', 'us politics', 'cricket', 'travel']
['football', 'stock market', 'science', 'style']
['cricket', 'science', 'tech']
['style', 'football', 'africa']
['stock market', 'coronavirus', 'asia']
['science', 'hollywood', 'us politics']
['us politics', 'cricket', 'africa', 'coronavirus']
['americas', 'asia', 'coronavirus']
['cricket', 'black lives matter', 'us politics', 'style']
['golf', 'tech', 'us politics']
['stock market', 'asia', 'football']
['asia', 'americas', 'stock market']
['europe', 'asia', 'science', 'americas']
['style', 'cricket',

In [ ]:
UserProfiler.head()

,UserId,SessionID,ArticleID Served,Article Rank,Click,Time Spent
0,0,0,607701c90b3e8bc9b7792dcb,0,1,51
1,0,0,6076ffa50b3e8bc9b7792c60,1,1,166
2,0,0,607700560b3e8bc9b7792cd3,2,1,166
3,0,0,607700a80b3e8bc9b7792d0b,3,0,-
4,0,0,6076fedc0b3e8bc9b7792bdf,4,0,-


In [ ]:
UserProfiler.shape

(60600, 6)

# Matrix Factorization

In [ ]:
def getNewsInfo(id):
  return data[data['_id']==id]

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
  
# Creating a user * news sparse matrix
sparseMatrix = csr_matrix((UserProfiler.UserId.unique().shape[0], data.shape[0])).toarray()

In [ ]:
k = 0
user = UserProfiler.iloc[k]

for i in UserProfiler.UserId.unique():
  while user.UserId == i and k < UserProfiler.shape[0]:
    user = UserProfiler.iloc[k]
    if user.Click:
      newsInfo = getNewsInfo(user['ArticleID Served'])
      rating = user['Time Spent']/newsInfo['Max_Time']
      sparseMatrix[i][newsInfo.index] = rating
    k+=1

In [ ]:
userItem = csr_matrix(sparseMatrix)
from numpy import count_nonzero
sparsity = 1.0 - count_nonzero(sparseMatrix) / sparseMatrix.size
print(sparsity)
pd.DataFrame(sparseMatrix)

0.9278818852924475


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1721,1722,1723,1724,1725,1726,1727,1728,1729,1730,1731,1732,1733,1734,1735,1736,1737,1738,1739,1740,1741,1742,1743,1744,1745,1746,1747,1748,1749,1750,1751,1752,1753,1754,1755,1756,1757,1758,1759,1760
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.424837,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.854839,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.696157,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.702703,0.000000,0.000000,0.000000,0.369775,0.356234,0.279097,0.000000,0.000000,0.822368,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.480263,0.000000,0.000000,0.00000,0.938940,0.000000,0.000000,0.000000,0.000000,0.338983,0.742237,0.000000,0.688679,0.808277,0.498325,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.487748,0.975894,0.940083,0.554054,0.886564,0.523392,0.850340,0.751608,0.000000,0.688836,0.774908,0.390909,0.449561,0.000000,0.000000,0.343137,0.759615,0.423554,0.000000,0.000000,0.000000,0.798969,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.763531,0.000000,0.617433,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.921053,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.945248,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.690789,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.

In [ ]:
def MF(X, num_dims, step_size,epochs,thres,lam_da):
  P = scipy.sparse.rand(X.shape[0], num_dims, 1, format='csr')
  P = scipy.sparse.csr_matrix(P/scipy.sparse.csr_matrix.sum(P, axis=1))
  Q = scipy.sparse.rand(num_dims, X.shape[1], 1, format='csr')
  Q = scipy.sparse.csr_matrix(Q/ scipy.sparse.csr_matrix.sum(Q, axis=0))

  prev_error = 0
  for iterat in range(epochs):
    errors = X - make_sparse(P.dot(Q).todense(), X)
    mse = np.sum(errors.multiply(errors))/len(X.indices)
    
    P_new=P + step_size*2*(errors.dot(Q.T)-lam_da*P)
    Q_new=Q + step_size*2*(P.T.dot(errors)-lam_da*Q)
    P=P_new
    Q=Q_new
    prev_error=mse
    #if iterat%1==0:
    print(iterat,mse)
  return pd.DataFrame(np.array(P.dot(Q).todense()))

    

In [ ]:
def make_sparse(array, X):
  array = np.array(array)
  x_pos, y_pos = X.nonzero()
  # print(x_pos, y_pos)
  try2 = np.array([[0 for i in range(array.shape[1])] for j in range(array.shape[0])])
  l = len(x_pos)
  for i in range(l):
    # print(x_pos[i], y_pos[i])
    try2[x_pos[i]][y_pos[i]] = array[x_pos[i]][y_pos[i]]

  return scipy.sparse.csr_matrix(try2)

In [ ]:
UserItem_MF = MF(userItem, 2, 0.005, 500, 0.00001, 5)

0 0.4203005053518187
1 0.3813844769604404
2 0.26646076205848224
3 0.36794417658168044
4 0.24793963125710902
5 0.38736795619341097
6 0.2371977986751857
7 0.3835936818864624
8 0.23083559527770095
9 0.38590473806563397
10 0.24017140374715454
11 0.38084394282022543
12 0.23705905093845792
13 0.38784965026013735
14 0.24535986917638794
15 0.38767465124091066
16 0.22961215129476065
17 0.3992676505477753
18 0.25286613171048017
19 0.38620225644131845
20 0.23282531192012987
21 0.403354700459861
22 0.24433842705253808
23 0.3985488633120374
24 0.22119685865700742
25 0.41785972618282524
26 0.2459119508502819
27 0.395486807633893
28 0.22474595677786666
29 0.4179801868629296
30 0.2388101596007939
31 0.398883016101095
32 0.224758148897861
33 0.41648054723298655
34 0.24086720778297127
35 0.39950944728768023
36 0.22352660916420136
37 0.41897333535667797
38 0.24425596124871388
39 0.39265227159883753
40 0.2249829005330648
41 0.4159515343556165
42 0.2443030182416885
43 0.39653534408344193
44 0.2218658060670

In [ ]:
def GetRecommendations(UserID, top_n = 10):
  userRating = UserItem_MF[UserID]
  s = numpy.array(userRating)
  sort_index = numpy.argsort(s)
  print(sort_index)
  print('Last n topics',data.iloc[sort_index[-top_n:]]['topics'])
  print('First n topics',data.iloc[sort_index[:top_n]]['topics'])
  recommended_ids = data.iloc[sort_index[-top_n:]]['_id']
  return recommended_ids

In [ ]:
ids = GetRecommendations(1)

[ 5 11 23 34 10 26 29 24 19 25 14 18  1  2  7  6 13 33 28 20 36  8  0 30
 16 17 37 39 21 32 22  3  4  9 31 38 35 12 27 15]
Last n topics 22    cricket
3     cricket
4     cricket
9     cricket
31    cricket
38    cricket
35    cricket
12    cricket
27    cricket
15    cricket
Name: topics, dtype: object
First n topics 5     cricket
11    cricket
23    cricket
34    cricket
10    cricket
26    cricket
29    cricket
24    cricket
19    cricket
25    cricket
Name: topics, dtype: object


In [ ]:
for id in ids:
  print(getNewsInfo(id))

    Unnamed: 0                       _id  ...     date_pub Max_Time
22          22  6076fafa0b3e8bc9b7792950  ...  Jul 6, 2020    228.4

[1 rows x 7 columns]
   Unnamed: 0                       _id  ...      date_pub Max_Time
3           3  6076fada0b3e8bc9b779293d  ...  Mar 27, 2021     56.0

[1 rows x 7 columns]
   Unnamed: 0                       _id  ...      date_pub Max_Time
4           4  6076fadb0b3e8bc9b779293e  ...  Feb 25, 2021    103.2

[1 rows x 7 columns]
   Unnamed: 0                       _id  ...      date_pub Max_Time
9           9  6076fae00b3e8bc9b7792943  ...  Jan 10, 2021    110.4

[1 rows x 7 columns]
    Unnamed: 0                       _id  ...      date_pub Max_Time
31          31  6076fb080b3e8bc9b7792959  ...  Mar 13, 2020    186.0

[1 rows x 7 columns]
    Unnamed: 0                       _id  ...     date_pub Max_Time
38          38  6076fb0f0b3e8bc9b7792960  ...  Jan 2, 2020    120.8

[1 rows x 7 columns]
    Unnamed: 0                       _id  ...     

# Implementing Flask API

In [ ]:
from flask import Flask
from flask_ngrok import run_with_ngrok
app = Flask(__name__)
run_with_ngrok(app)   
  
@app.route("/")
def home():
    return "<h1>GFG is great platform to learn</h1>"
    
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://136eb9c51265.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [14/Apr/2021 20:42:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2021 20:42:11] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
import numpy
s = numpy.array([2, 3, 1, 4, 5])
sort_index = numpy.argsort(s)
print(sort_index)

[2 0 1 3 4]
